In [1]:
from pyspark.sql.functions import monotonically_increasing_id, col, to_date

# 1. Load Silver layer (your one big fact file)
df = spark.read.format("delta").load("abfss://Project_WS@onelake.dfs.fabric.microsoft.com/Silver_Lakehouse.Lakehouse/Tables/dbo/fact_user_engagament_table")

# 2. Clean & convert date fields
df = df.withColumn("web_visit_date", to_date("web_visit_date", "yyyy-MM-dd"))
df = df.withColumn("social_date", to_date("social_date", "yyyy-MM-dd"))
df = df.withColumn("demand_date", to_date("demand_created_date", "yyyy-MM-dd"))

# =========================
# 3. Build Dimension Tables
# =========================

# 3.1 dim_user
dim_user = df.select("user_id", "first_name", "last_name", "account_email").dropDuplicates()
dim_user = dim_user.withColumn("user_key", monotonically_increasing_id())
dim_user = dim_user.select("user_key", "user_id", "first_name", "last_name", col("account_email").alias("email"))
dim_user.write.mode("overwrite").format("delta").save("Files/Dim_user_file/dim_user")

# 3.2 dim_product
dim_product = df.select("product_id", "product_name", "category", "location").dropDuplicates()
dim_product = dim_product.withColumn("product_key", monotonically_increasing_id())
dim_product.write.mode("overwrite").format("delta").save("Files/Dim_product_file/dim_product")

# 3.3 dim_platform
dim_platform = df.select("platform_name", "device").dropDuplicates()
dim_platform = dim_platform.withColumn("platform_key", monotonically_increasing_id())
dim_platform.write.mode("overwrite").format("delta").save("Files/Dim_platform-file/dim_platform")

# 3.4 dim_date (unified from all 3 date columns)
date_union = df.select(col("web_visit_date").alias("date")) \
    .union(df.select(col("social_date").alias("date"))) \
    .union(df.select(col("demand_date").alias("date"))) \
    .dropDuplicates()

dim_date = date_union.withColumn("date_key", monotonically_increasing_id()) \
    .select("date_key", "date")

dim_date.write.mode("overwrite").format("delta").save("Files/Dim_date_file/dim_date")

# =========================
# 4. Build Fact Table
# =========================

# Join with dim_user
fact = df.join(dim_user, on="user_id", how="left")

# Join with dim_product
fact = fact.join(dim_product, on=["product_id", "product_name", "category", "location"], how="left")

# Join with dim_platform
fact = fact.join(dim_platform, on=["platform_name", "device"], how="left")

# Final fact table selection with surrogate keys
fact_final = fact.select(
    "user_key",
    "product_key",
    "platform_key",
    "web_visit_date",
    "social_date",
    "demand_date",
    "status",
    "interaction_type"
)

# Save to Gold
fact_final.write.mode("overwrite").format("delta").save("Files/Fact_user_engagement_file/fact_user_engagement")


StatementMeta(, 964243ff-6f60-47cf-a83e-9e39c3ad96e9, 3, Finished, Available, Finished)